In [83]:
import nltk
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\José
[nltk_data]     Neto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Carregar dataset

In [84]:
# Carregar dataset
df = pd.read_csv('../datasets/dataset-case_study.csv')

# Informações sobre o conjunto de dados
print('Dataset antes da limpeza:')
print(df.shape)
print(df.columns)

# Contar quantas linhas tem para cada plataforma
platform_counts = df['platform'].value_counts()
print("\nDistribuição por plataforma:", platform_counts)

Dataset antes da limpeza:
(5933, 9)
Index(['restaurante_name', 'user_name', 'review_date', 'stars', 'comment_text',
       'year', 'platform', 'language', 'comment_text_translated'],
      dtype='object')

Distribuição por plataforma: platform
Google             3917
Facebook            847
Foursquare          834
Yelp                310
Zomato               24
Restaurant Guru       1
Name: count, dtype: int64


### Limpeza dos dados

In [85]:
def remove_empty_text_rows(df):
    empty_sentenca_rows = df['comment_text_translated'].isnull() | (df['comment_text_translated'] == '')
    # Imprimir o número de sentenças vazias
    num_empty_sentences = empty_sentenca_rows.sum()
    print(f"Total de comentários vazios: {num_empty_sentences}")
    print('Removendo comentários vazios...\n')
    # Remover sentenças vazias
    df = df[~empty_sentenca_rows]
    df.reset_index(drop=True, inplace=True)
    return df

In [86]:
def split_sentences(df):
    rows = []
    for index, row in df.iterrows():
        sentences = row['comment_text_translated'].split('.')
        for sentence in sentences:
            sentence = sentence.strip()
            if sentence:  # apenas sentenças não vazias
                new_row = row.copy()
                new_row['comment_text_translated'] = sentence
                rows.append(new_row)
    return pd.DataFrame(rows)

In [87]:
# Remover sentenças nulas
df = remove_empty_text_rows(df)

# Separar os comentários em sentenças
df = split_sentences(df)

Total de comentários vazios: 3
Removendo comentários vazios...



In [88]:
# Substituir espaços múltiplos por um único espaço e realizar strip
df['comment_text_translated'] = df['comment_text_translated'].astype(str).apply(lambda x: ' '.join(x.split()))

# Remover ocorrências de erros
replacements = {
    'Comida': 'comida',
    'Serviço': 'serviço',
    'Atmosfera': 'atmosfera',
    'Food': 'food',
    'Service': 'service',
    'Atmosphere': 'atmosphere'
}

for uppercase, lowercase in replacements.items():
    df['comment_text_translated'] = df['comment_text_translated'].str.replace(uppercase, lowercase, regex=False)

replacements = [
    'comida: 5', 'food: 5', 'comida: 4', 'food: 4', 'comida: 3', 'food: 3', 'comida: 2', 'food: 2', 'comida: 1', 'food: 1',
    'serviço: 5', 'service: 5', 'serviço: 4', 'service: 4', 'serviço: 3', 'service: 3', 'serviço: 2', 'service: 2', 'serviço: 1', 'service: 1',
    'atmosfera: 5', 'atmosphere: 5', 'atmosfera: 4', 'atmosphere: 4', 'atmosfera: 3', 'atmosphere: 3', 'atmosfera: 2', 'atmosphere: 2', 'atmosfera: 1', 'atmosphere: 1',
    ': 5', ': 4', ': 3', ': 2', ': 1'
]

for r in replacements:
    df['comment_text_translated'] = df['comment_text_translated'].str.replace(r, '')

# Substituir espaços múltiplos por um único espaço e realizar strip
df['comment_text_translated'] = df['comment_text_translated'].astype(str).apply(lambda x: ' '.join(x.split()))

# Remover sentenças nulas
df = remove_empty_text_rows(df)

# Remover sentenças duplicadas
comentarios_duplicados = df.duplicated(subset="comment_text_translated").sum()
print("Total de comentários duplicados:", comentarios_duplicados)
print('Removendo comentários duplicados...\n')
df = df.drop_duplicates(subset="comment_text_translated", keep='first').reset_index(drop=True)

Total de comentários vazios: 531
Removendo comentários vazios...

Total de comentários duplicados: 2101
Removendo comentários duplicados...



### Stemming

In [89]:
# Realizar Stemming
stemmer = PorterStemmer()
def stem_words(sentence):
    words = word_tokenize(sentence)
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

df['comment_text_translated'] = df['comment_text_translated'].apply(stem_words)

### Dataset Final

In [90]:
# Informações sobre o conjunto de dados
print('Dataset após limpeza:')
print(df.shape)

# Contar quantas linhas tem para cada plataforma
platform_counts = df['platform'].value_counts()
print("\nDistribuição por plataforma:", platform_counts)

Dataset após limpeza:
(13733, 9)

Distribuição por plataforma: platform
Google             8322
Yelp               2345
Facebook           1559
Foursquare         1368
Zomato              124
Restaurant Guru      15
Name: count, dtype: int64


In [91]:
name = 'teste2'
df.to_csv(f'../datasets/dataset-case_study-{name}.csv', index=False)